In [2]:
import requests
from bs4 import BeautifulSoup
 
import pandas as pd
import re
from datetime import datetime
import time
from tqdm import tqdm
 
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import urllib.request

from io import BytesIO
# from openpyxl import Workbook
# from openpyxl.drawing.image import Image

from upload_data import *

options = webdriver.ChromeOptions()
# options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("window-size=2560x1440")
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
# options.add_argument('--headless')  # 브라우저를 숨기는 옵션 추가

In [3]:
data = pd.read_parquet("C:/Users/tips_workstation2/Downloads/meta_출산-유아용품_50000614_20240816-18_13_24.parquet")

In [14]:
data.to_csv('C:/Users/tips_workstation2/Downloads/meta_출산-유아용품_50000614_20240816-18_13_24.csv.', encoding = 'utf-8-sig')

In [10]:
# pip install openpyxl

In [5]:
data = data.drop_duplicates(subset= ['product_names','product_urls'])

In [6]:
len(data)

120

In [7]:
product_urls = data['product_urls'].to_list()

In [12]:
len(product_urls)

120

In [24]:
# except_data = pd.read_csv("C:/Users/tips_workstation2/Desktop/C_Review/df_except_url.csv")
# except_data = except_data['except_url'].to_list()

In [20]:
chrome_driver_path = r'C:/Users/tips_workstation2/Desktop/new_crawler/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
# urls = ['https://cr.shopping.naver.com/adcr.nhn?x=DwrCo1vLLbjpUak5juPgtf%2F%2F%2Fw%3D%3DsZtY5NFjXAfsivixZsTQomb%2BcLFA0Qj567HSfc9VC02OedMUWqnXIzj66FooS5oY%2FjBHJ8G0YZYRcccyU4ef9Gf8FHPDjn2BfnATINGE76fDbec32ynutqa9vppdSodNmg4Jh40wBIvXqZKMKJ%2BTVzMu1rI7y%2Bnxga7qkc20jAaEHS16Mm4swTlRWNG5xgFwbbzflVy0wI0y%2FZd91tQThG5vMghwznzbbU4PEkj4x778V0MIubreMfYn18wDP4TWofwtKKs9UXD0b%2BuMkAbBKR6Tq%2FbD2MGowZHhNOyBhpCeUgj7JhipZW7dJJfFyMQl0M6N8r%2B4R2IFQhbBQnBk56Vpd6HkF4PlrVyRjT1YfTWzXu9%2FbTMMj8ShqDHul8W1pZFb3q7ed3JQofxiIa%2F5WUUwWXwjJy%2Ff3jjuntaq%2BPYVx3qD225EaXn4cT7PNXiT4OiwvCtxrEMi02%2Brr8wku4zdQzQ6h%2Bx0nC1fw5saLnpP7mznP4AC3nBVLt9BYjX0xINy%2BaWdOz5NwdOp8Vl8G9KKqzblhPlZKVIDKnHwXy02MmdE970Z%2BYe%2Bg4Nl6BjN1sG3XYgKyOfWh13MoBcHdJ1VnRdYExPsTAl3EvuAdwXtTMmE6D6k7Fn1YO%2BPHYVGk9bCYs7o1a2MZxG3O5EqLl5y8sARekz3cqx4uaUskJFYNUJgJp6CUXauCvW7Y5AjN&nvMid=86362778320&catId=50003455']
urls = product_urls

exception_urls = []

# dataframes = {}
# folder_paths = []

# 통합리스트 초기화
all_brand_names = []
all_product_names = []
all_write_dt_lst = []
all_rating_lst = []
all_content_lst = []

count = 0
# URL에서 데이터 수집
for url in tqdm(urls):
    driver.get(url)
    actual_url = driver.current_url
    print(actual_url)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 현재 url이 두가지로 시작하지 않으면 예외처리 후 다음 url로 넘김
    if not (actual_url.startswith('https://brand') or actual_url.startswith('https://smartstore')) :
        exception_urls.append(actual_url)
        continue

    # 각 url 마다 수집할 list 초기화
    brand_name = []
    product_name = []
    write_dt_lst = []
    rating_lst = []
    content_lst = []

    # 브랜드명과 상품명을 수집 -> 사이트마다 브랜드명이 들어가는 방법이 크게 4가지로 보임
    # try: 
    #     try:
    #         brand_element = driver.find_element(By.CSS_SELECTOR, '#pc-storeNameWidget > div > div > a > img')
    #         brand = brand_element.get_attribute('alt')
    #     except:
    #         try:
    #             brand_element = driver.fine_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > img")
    #             brand = brand_element.get_attribute('alt')
    #         except:
    #             try:
    #                 brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-storeNameWidget > div > div > a > span")
    #                 brand = brand_element.text
    #             except:   
    #                 brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > span")
    #                 brand = brand_element.text
    #                 try :
    #                     brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > img")
    #                     brand = brand_element.get_attribute('alt')
    #                 except Exception as e:
    #                     print(f"브랜드명 수집 실패 : {e}")
        
    #     #content > div > div._2-I30XS1lA > div._2QCa6wHHPy > fieldset > div._3k440DUKzy > div._1eddO7u4UC > h3
    #     # product = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[2]/div[2]/fieldset/div[1]/div[1]/h3').text
    #     product = driver.find_element(By.CSS_SELECTOR, '#content > div > div._2-I30XS1lA > div._2QCa6wHHPy > fieldset > div._3k440DUKzy > div._1eddO7u4UC > h3').text
    #     # driver.execute_script("arguments[0].scrollIntoView(true);", product)
    # except Exception as e:
    #     print(f"상품명 수집 실패 : {e}")
    #     exception_urls.append(url)
    #     continue
     #pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > img

 
    try:
        brand_element = driver.find_element(By.CSS_SELECTOR, '#pc-storeNameWidget > div > div > a > img')
        brand = brand_element.get_attribute('alt')
    except:
        try:
            brand_element = driver.fine_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > img")
            brand = brand_element.get_attribute('alt')
        except:
            try:
                brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-storeNameWidget > div > div > a > span")
                brand = brand_element.text
            except:   
                brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > span")
                brand = brand_element.text
                try :
                    brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > img")
                    brand = brand_element.get_attribute('alt')
                except Exception as e:
                    print(f"브랜드명 수집 실패 : {e}")
    
    # 마우스 scroll을 통해 가장 마지막 페이지까지 내려줌
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(10)
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        if scroll_location == scroll_height:
            break
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")

    try:
        product = driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div.XYLPnZFSI9 > strong').text

    except Exception as e:
        print(f"상품명 수집 실패 : {e}")
        exception_urls.append(url)
        continue

    # 긁은 상품명에 마지막에 띄어쓰기가 들어가는 경우 경로를 찾지 못하는 에러 발생함
    sanitized_product = product.replace("/", "_").replace(" ", "_").replace(":", "_").replace("+", "_")
    folder_name = sanitized_product
    # folder_name = f"{product}"
    folder_path = os.path.join(f'C:/Users/tips_workstation2/Desktop/C_Review/', folder_name) # local_path
    os.makedirs(folder_path, exist_ok=True)

    try:
        # 리뷰 버튼 클릭
        #_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(2) > a 
        #content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a
        try : 
            driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(2) > a').click()
            time.sleep(3)
        except :
            driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
            time.sleep(3)

        # 최신순 버튼 클릭  
        driver.find_element(By.CSS_SELECTOR, '#REVIEW > div > div._2LvIMaBiIO > div._2LAwVxx1Sd > div._1txuie7UTH > ul > li:nth-child(2) > a').click()
        time.sleep(3)

    # 리뷰 버튼이나 최신순 버튼이 없으면 리뷰가 없을 가능성이 높음으로 다음 url로 넘어가게 함
    except Exception as e:
        print(f"최신순 버튼 클릭 실패: {e}")
        exception_urls.append(actual_url)
        continue

    # 현재 페이지
    page_num = 1
    # 다음 버튼 css_element의 nth_child({page_ctl}) 부분
    page_ctl = 3

    while True:
        print(f'start: {page_num} page 수집 중, page_ctl: {page_ctl}')

        # 리뷰가 있는지 확인
        reviews = soup.findAll('li', {'class': 'BnwL_cs1av _nlog_click _nlog_impression_element'})

        # 최신순 버튼은 눌렀으나 여기에 리뷰가 없을 가능성을 생각해서 로직 하나 추가
        if not reviews:
            print("리뷰가 없습니다. 다음 URL로 넘어갑니다.")
            exception_urls.append(actual_url)
            break 

        for review in range(len(reviews)):
            # 4-1. 리뷰 작성 일자 수집
            write_dt_raw = reviews[review].findAll('span', {'class': '_2L3vDiadT9'})[0].get_text()
            write_dt = datetime.strptime(write_dt_raw, '%y.%m.%d.').strftime('%Y%m%d')

            # 4-2. 평점 수집
            rating = reviews[review].findAll('em', {'class': '_15NU42F3kT'})[0].get_text()

            # 4-3. 리뷰 내용 수집
            review_content_raw = reviews[review].findAll('div', {'class': '_1kMfD5ErZ6'})[0].find('span', {'class': '_2L3vDiadT9'}).get_text()
            review_content = re.sub(' +', ' ', re.sub('\n', ' ', review_content_raw))

            # 4-4. 리뷰 이미지 수집 (첫 페이지만 수집)
            if page_num == 1:
                # review_image_tag = reviews[review].find('img', {'class': '_3Lp-477Dqi'})
                review_image_tag = reviews[review].find('span', {'class': '_1DOkWFrX74'})               
                if review_image_tag:
                    try : 
                        review_image_tags = review_image_tag.find_all('img')
                        if review_image_tags:                          
                            review_image_link = review_image_tags[0]['data-src']
                            print(f'Review Image Link : {review_image_link}')
                            review_image_link = review_image_link.replace('https', 'http')
                            opener = urllib.request.build_opener()
                            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
                            urllib.request.install_opener(opener)
                            local_img_path = os.path.join(folder_path, f'{count}.jpg')
                            urllib.request.urlretrieve(review_image_link, local_img_path) # local 저장 path
                            count += 1

                            # s3에도 업로드
                            ecommerce = 'naver'
                            crawling_type = 'review'
                            category = '출산-육아용품'         # 여기부터 변경
                            search_type = 'ctg4_ID'
                            file_type = 'image'
                            keyword = '50000614'
                            jpg_fullname = f"{folder_name}/{count}"
                            jpg_file_path = local_img_path
                            
                            save_review_image_data(ecommerce, crawling_type, category, search_type, file_type, jpg_file_path, jpg_fullname, keyword)
                            
                    except Exception as e:
                        print(f"src 수집 실패 : {e}")
                        break
            else:
                review_image = ''

            # 4-5. 수집 데이터 리스트 저장
            brand_name.append(brand)
            product_name.append(product)    
            write_dt_lst.append(write_dt)
            rating_lst.append(rating)
            content_lst.append(review_content)

        # 크롤링 중지 기준 날짜 설정
        # 리뷰 수집 일자 기준 데이터 확인(페이지 긁고 최근 6개월 날짜 이전이면 break)
        date_cut = (datetime.now() - timedelta(days=180)).strftime('%Y%m%d')
        if write_dt_lst[-1] < date_cut:
            break

        # 리뷰 수집이 덜 되었으면 다음 페이지로 넘어가서 다시 긁음
        # brandstore의 경우 페이지 넘김 부분이 10 페이지 이후에 다음 버튼을 한번 눌러줘야 함
        if actual_url.startswith('https://brand'):
            if page_num % 10 == 0:
                # 다음 버튼 클릭
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, "#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > div > div > a.fAUKm1ewwo._2Ar8-aEUTq._nlog_click")
                    next_button.click()
                    time.sleep(3)  # 페이지 로딩 시간을 고려하여 잠시 대기
                except Exception as e:
                    print(f"다음 버튼 클릭 실패: {e}")
                    break
                # page_ctl 초기화
                page_ctl = 3
            else:
                try:
                    driver.find_element(By.CSS_SELECTOR, f'#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > div > div > a:nth-child({page_ctl})').click()
                    time.sleep(3)
                except Exception as e:
                    print(f"페이지 버튼 클릭 실패: {e}")
                    break
                page_ctl += 1
        # smartstore의 경우 페이지 넘김 부분이 6 페이지 이후 고정되기 때문에, 계속 다음 버튼만 누름
        elif actual_url.startswith('https://smartstore'):
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > div > div > a.fAUKm1ewwo._2Ar8-aEUTq._nlog_click")
                next_button.click()
                time.sleep(4)  # 페이지 로딩 시간을 고려하여 잠시 대기
            except Exception as e:
                print(f"다음 버튼 클릭 실패: {e}")
                break

        page_num += 1

    # # 각 url마다 수집되는 list들을 df로 생성
    # df = pd.DataFrame({
    #     'brand_name': brand_name,
    #     'product_name': product_name,
    #     'write_dt_lst': write_dt_lst,
    #     'rating_lst': rating_lst,
    #     'content_lst': content_lst
    # })

    # 구문 가장 밖에 있는 list에 모든 값들을 extend -> url 순서대로 모든 정보가 차곡자곡 쌓임
    all_brand_names.extend(brand_name)
    all_product_names.extend(product_name)
    all_write_dt_lst.extend(write_dt_lst)
    all_rating_lst.extend(rating_lst)
    all_content_lst.extend(content_lst)

# 모아진 모든 list를 df화
df_all = pd.DataFrame({
    'brand_name' : all_brand_names,
    'product_name' : all_product_names,
    'write_dt_lst' : all_write_dt_lst,
    'rating_lst' : all_rating_lst,
    'content_lst' : all_content_lst
})

# ====================save to S3 folder=========================
# 카테고리마다 변경
ecommerce = 'naver'
crawling_type = 'review'
category = '출산-육아용품'
search_type = 'ctg4_ID' # or 'ctg4_ID'
file_type = 'reviews'
keyword = '50000614'
save_review_data(df_all, ecommerce, crawling_type, category, search_type, file_type, keyword)

print('done')
driver.quit() 


if exception_urls:
    print(f"예외 URL ({len(exception_urls)}개):")
    for exception_url in exception_urls:
        print(exception_url)
else:
    print("예외 URL 없음")

df_except_url = pd.DataFrame({
    'except_url' : exception_urls})

df_except_url.to_csv(f'C:/Users/tips_workstation2/Desktop/C_Review/df_except_url_{keyword}.csv')


In [11]:
df_all

,brand_name,product_name,write_dt_lst,rating_lst,content_lst
0,질마재 농장,[질마재농장]유기농 쌀과자 백미 시리얼 120g 외11종 [원산지:국산(전라북도 고...,20240816,4,고소하고 맛있어요!
1,질마재 농장,[질마재농장]유기농 쌀과자 백미 시리얼 120g 외11종 [원산지:국산(전라북도 고...,20240816,4,고소하고 맛있어요!
2,질마재 농장,[질마재농장]유기농 쌀과자 백미 시리얼 120g 외11종 [원산지:국산(전라북도 고...,20240815,5,다른 첨가물도 없고 곡물 그 자체라 좋아요. 아이도 잘 먹어서 여러개 쟁여놨어요~ㅎ
3,질마재 농장,[질마재농장]유기농 쌀과자 백미 시리얼 120g 외11종 [원산지:국산(전라북도 고...,20240815,5,첨가물도 없고 곡물 그 자체라 좋아요. 아이도 잘먹어요ㅎ
4,질마재 농장,[질마재농장]유기농 쌀과자 백미 시리얼 120g 외11종 [원산지:국산(전라북도 고...,20240815,5,다른것보다 유난히 이걸좋아하네요
...,...,...,...,...,...
52628,그린원푸드,그린원푸드 유기농 옹알이 쌀과자 30g 5+1 (1개랜덤) [원산지:국산],20240202,5,종류별로 구매했어요 다먹으면 재구매하려구요
52629,그린원푸드,그린원푸드 유기농 옹알이 쌀과자 30g 5+1 (1개랜덤) [원산지:국산],20240202,5,아기가 좋아해요 개인적으로 단호박 흑임자가 좋아요
52630,그린원푸드,그린원푸드 유기농 옹알이 쌀과자 30g 5+1 (1개랜덤) [원산지:국산],20240202,5,유기농에 다른 첨가물 없이 건강한 맛이예요
52631,그린원푸드,그린원푸드 유기농 옹알이 쌀과자 30g 5+1 (1개랜덤) [원산지:국산],20240202,5,지인에게 받아 한봉 먹여봤는데 아기도 잘먹고 한입 쏙이라 손이 지저분해지지 않아 좋아요
